In [ ]:
# === CONFIGURATION ===
TOPO = "BA"   # "BA" oR "ER"
Z_VAL = 16     # 4 or 16
N = 500
REPS = 100
STEPS = 15000
FILE_NAME = f"DATA_{TOPO}_Z{Z_VAL}.csv"
# =====================

import numpy as np
import networkx as nx
import pandas as pd
import os, time
from collections import deque
from numba import njit

# -------------------------
# 1) Build graph
# -------------------------
def build_game_graph(topo: str, N: int, z: int, seed: int = 42):
    if topo == "BA":
        G = nx.barabasi_albert_graph(N, int(z/2), seed=seed)
    else:
        G = nx.erdos_renyi_graph(N, z/(N-1), seed=seed)
    return G

def graph_to_neighbor_arrays(G: nx.Graph):
    N = G.number_of_nodes()
    degs = np.array([G.degree(i) for i in range(N)], dtype=np.int32)
    max_deg = int(degs.max())
    neigh_idx = np.empty((N, max_deg), dtype=np.int32)
    neigh_cnt = np.empty(N, dtype=np.int32)

    for i in range(N):
        nbrs = list(G.neighbors(i))
        neigh_cnt[i] = len(nbrs)
        # pad quelconque (ne sera pas lu au-delà de neigh_cnt)
        neigh_idx[i, :len(nbrs)] = np.array(nbrs, dtype=np.int32)
        if len(nbrs) < max_deg:
            neigh_idx[i, len(nbrs):] = 0

    return neigh_idx, neigh_cnt, degs.astype(np.float64)

# -------------------------
# 2) Influence graph + BFS
# -------------------------
def relabel_graph_with_fixed_permutation(G: nx.Graph, seed: int = 123):
    rng = np.random.default_rng(seed)
    nodes = np.array(G.nodes(), dtype=np.int32)
    perm = nodes.copy()
    rng.shuffle(perm)
    mapping = dict(zip(nodes, perm))
    return nx.relabel_nodes(G, mapping)

def build_influence_layers(G_infl: nx.Graph, Lmax: int):

    N = G_infl.number_of_nodes()
    inf_indices = np.zeros((Lmax, N, N), dtype=np.int32)
    inf_counts  = np.zeros((Lmax, N), dtype=np.int32)

    # BFS tronqué à Lmax depuis chaque i
    adj = [list(G_infl.neighbors(i)) for i in range(N)]

    for i in range(N):
        dist = np.full(N, -1, dtype=np.int32)
        dist[i] = 0
        q = deque([i])

        while q:
            u = q.popleft()
            du = dist[u]
            if du == Lmax:
                continue
            for v in adj[u]:
                if dist[v] == -1:
                    dist[v] = du + 1
                    q.append(v)

        # remplir par couche
        for j in range(N):
            d = dist[j]
            if 0 < d <= Lmax:
                d_idx = d - 1
                c = inf_counts[d_idx, i]
                inf_indices[d_idx, i, c] = j
                inf_counts[d_idx, i] += 1

    return inf_indices, inf_counts

# -------------------------
# 3) Numba: simulaciones
# -------------------------
@njit
def run_simulation_fast(neigh_idx, neigh_cnt, degrees,
                        inf_indices, inf_counts, alpha_weights,
                        theta, b, steps, seed):
    N = neigh_cnt.shape[0]
    np.random.seed(seed)

    # allocations uniques
    strat = np.random.randint(0, 2, N).astype(np.float64)
    vig = np.zeros(N, dtype=np.float64)

    # init vig (comme toi)
    for i in range(N):
        if strat[i] == 1.0 and np.random.random() < 0.5:
            vig[i] = 1.0

    L = alpha_weights.shape[0]
    influences = np.zeros(N, dtype=np.float64)
    payoffs = np.zeros(N, dtype=np.float64)
    new_strat = np.empty(N, dtype=np.float64)
    T_eff = np.empty(N, dtype=np.float64)

    # stabilité: variance glissante sur fenêtre
    win = 300
    buf = np.zeros(win, dtype=np.float64)
    buf_pos = 0
    buf_full = 0

    burnin = 800
    check_every = 50
    eps_std = 2e-4  
    
    for t in range(steps):
        # A) Influence
        for i in range(N):
            I_i = 0.0
            for l in range(L):
                cnt = inf_counts[l, i]
                if cnt > 0:
                    s = 0.0
                    for k in range(cnt):
                        s += vig[inf_indices[l, i, k]]
                    I_i += alpha_weights[l] * (s / cnt)
            influences[i] = I_i

        # B) Vigilance
        for i in range(N):
            if strat[i] == 1.0:
                vig[i] = 1.0 if influences[i] >= theta else 0.0
            else:
                vig[i] = 0.0

        # C) Payoffs
        # T_eff(i) = 1 + (b-1)*(1-influence)
        for i in range(N):
            T_eff[i] = 1.0 + (b - 1.0) * (1.0 - influences[i])

        for i in range(N):
            payoffs[i] = 0.0
            ni = neigh_cnt[i]
            for kk in range(ni):
                j = neigh_idx[i, kk]
                if strat[i] == 1.0:
                    if strat[j] == 1.0:
                        payoffs[i] += 1.0
                else:
                    if strat[j] == 1.0:
                        payoffs[i] += T_eff[i]

        # D) Imitation
        for i in range(N):
            new_strat[i] = strat[i]

        for i in range(N):
            ni = neigh_cnt[i]
            if ni > 0:
                j = neigh_idx[i, np.random.randint(0, ni)]
                if payoffs[j] > payoffs[i]:
                    # phi = max(deg_i, deg_j) * max(Teff_i, Teff_j)
                    di = degrees[i]
                    dj = degrees[j]
                    phi = (di if di > dj else dj) * (T_eff[i] if T_eff[i] > T_eff[j] else T_eff[j])
                    if phi > 0.0:
                        if np.random.random() < (payoffs[j] - payoffs[i]) / phi:
                            new_strat[i] = strat[j]

        # swap
        for i in range(N):
            strat[i] = new_strat[i]

        rho = 0.0
        for i in range(N):
            rho += strat[i]
        rho /= N

        # absorption
        if rho <= 0.0 or rho >= 1.0:
            # mean vig
            mv = 0.0
            for i in range(N):
                mv += vig[i]
            return rho, mv / N

        # update buffer
        buf[buf_pos] = rho
        buf_pos += 1
        if buf_pos == win:
            buf_pos = 0
            buf_full = 1

        # check stability (std window)
        if t > burnin and (t % check_every == 0) and buf_full == 1:
            m = 0.0
            for k in range(win):
                m += buf[k]
            m /= win
            v = 0.0
            for k in range(win):
                d = buf[k] - m
                v += d * d
            v /= win
            std = np.sqrt(v)
            if std < eps_std:
                mv = 0.0
                for i in range(N):
                    mv += vig[i]
                return m, mv / N

    # end
    if buf_full == 1:
        m = 0.0
        for k in range(win):
            m += buf[k]
        m /= win
    else:
        m = rho

    mv = 0.0
    for i in range(N):
        mv += vig[i]
    return m, mv / N

# -------------------------
# 4) Wrapper: 100 seeds 
# -------------------------
def run_reps_fixed_graph(neigh_idx, neigh_cnt, degrees,
                         inf_indices_L, inf_counts_L, alpha_weights,
                         theta, b, steps, reps, base_seed):
    rhos = np.empty(reps, dtype=np.float64)
    vigs = np.empty(reps, dtype=np.float64)
    for r in range(reps):
        rho, v = run_simulation_fast(
            neigh_idx, neigh_cnt, degrees,
            inf_indices_L, inf_counts_L, alpha_weights,
            theta, b, steps, base_seed + r
        )
        rhos[r] = rho
        vigs[r] = v
    return rhos, vigs

# ============================================================
# MAIN
# ============================================================
G_game = build_game_graph(TOPO, N, Z_VAL, seed=42)
neigh_idx, neigh_cnt, degrees = graph_to_neighbor_arrays(G_game)

G_infl_max = G_game
G_infl_nul = relabel_graph_with_fixed_permutation(G_game, seed=123)

# pre calc
Lmax = 4
inf_idx_max, inf_cnt_max = build_influence_layers(G_infl_max, Lmax=Lmax)
inf_idx_nul, inf_cnt_nul = build_influence_layers(G_infl_nul, Lmax=Lmax)

# simualation
for corr in ["max", "nulle"]:
    for theta in [0.2, 0.4, 0.6]:
        for L in [1, 4]:
            alphas = np.array([0.5**l for l in range(L)], dtype=np.float64)
            alphas /= np.sum(alphas)

            if corr == "max":
                inf_indices_L = inf_idx_max[:L].copy()
                inf_counts_L  = inf_cnt_max[:L].copy()
                base_seed = 10000
            else:
                inf_indices_L = inf_idx_nul[:L].copy()
                inf_counts_L  = inf_cnt_nul[:L].copy()
                base_seed = 20000

            for b in np.around(np.arange(1.0, 2.1, 0.1), 1):
                start = time.time()

                rhos, vigs = run_reps_fixed_graph(
                    neigh_idx, neigh_cnt, degrees,
                    inf_indices_L, inf_counts_L, alphas,
                    theta, float(b), STEPS, REPS,
                    base_seed=base_seed + int(theta*100) + L*1000 + int(b*10)*10
                )

                df = pd.DataFrame([{
                    "Corr": corr, "Theta": theta, "L": L, "b": b,
                    "C_mean": float(np.mean(rhos)), "C_std": float(np.std(rhos)),
                    "V_mean": float(np.mean(vigs)), "V_std": float(np.std(vigs))
                }])

                df.to_csv(FILE_NAME, mode='a', header=not os.path.exists(FILE_NAME), index=False)
                print(f"Done: {corr}, Th={theta}, L={L}, b={b} ({time.time()-start:.1f}s)")
